### Load Pre-Processed Data

In [11]:
# retrieve stored values

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le


### Setting up CNN

In [12]:
disable_gpu = False

# To disable gpu if wanted
if disable_gpu:
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
import tensorflow as tf

if len(tf.config.list_physical_devices('GPU')):
    print("GPU detected")
else:
    print("CPU detected, this may take a while")

GPU detected


In [13]:
# possible way to free up gpu memory after easier https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/discussion/96876

num_rows = 40
num_columns = 174
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax')) 

### Compiling Model

In [14]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [15]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 39, 173, 16)       80        
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 19, 86, 16)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 19, 86, 16)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 18, 85, 32)        2080      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 9, 42, 32)        0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 9, 42, 32)        

In [16]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
#import tqdm.notebook as tqdm
import tqdm
from tqdm.keras import TqdmCallback

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='models/weights.best.basic_cnn.hdf5', 
                               verbose=0, save_best_only=True)
start = datetime.now()
    
history = model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer,TqdmCallback(verbose=1)], verbose = 0)

duration = datetime.now() - start
print("Training completed in time: ", duration)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

2023-07-01 16:54:57.833786: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Training completed in time:  0:01:57.607667


### Results from training


In [19]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9928207993507385
Testing Accuracy:  0.9360047578811646


### Save model for use elsewhere

In [20]:
import pickle

model.save('sound_category_model.h5')

# sound_model_file = "sound_model.pkl"

# with open(sound_model_file, 'wb') as file:
#     pickle.dump(model, file)

label_encoder_file = "labelencoder.pkl"


    
with open(label_encoder_file, 'wb') as file:
    pickle.dump(le, file)

#%store model
